In [10]:
import os
import keras
import numpy as np
training_folder_name = 'data/shapes/training'
classes = os.listdir(training_folder_name)
classes

['circle', 'square', 'triangle']

In [11]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)

img_size = (128,128)
batch_size = 30

train_generator = datagen.flow_from_directory(training_folder_name,
                                              target_size=img_size,
                                              batch_size=batch_size,class_mode="categorical",subset="training" )

validation_generator = datagen.flow_from_directory(training_folder_name,
                                              target_size=img_size,
                                              batch_size=batch_size,class_mode="categorical",subset="validation" )

Found 840 images belonging to 3 classes.
Found 360 images belonging to 3 classes.


In [32]:
print("Image Shape :",train_generator.image_shape)
print("Number of Classses :",train_generator.num_classes)
print("Train Sample :", train_generator.samples)
print("Validation Sample :", validation_generator.samples)

Image Shape : (128, 128, 3)
Number of Classses : 3
Train Sample : 840
Validation Sample : 360


In [33]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, Dense
from keras.optimizers import Adam

filter_size = (6,6) # also called as kernal size
# 32 is number of unit in first convolution layer
input_shape = train_generator.image_shape # (128 x 128 x 3)
no_classes = train_generator.num_classes # which is 3(circle, square, triangle)
activation_1, activation_2 = "relu", "softmax" #Using Rectified Linear Unit and Softmax for this model


# Model Creatation
model = Sequential()

model.add(Conv2D(32, (6,6), input_shape=input_shape, activation=activation_1 ))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (6,6), activation= activation_1))

model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(no_classes, activation=activation_2))

opt = Adam(lr=0.001)

model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
          
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 123, 123, 32)      3488      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 61, 61, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 56, 56, 32)        36896     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 28, 28, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 75267     
Total params: 115,651
Trainable params: 115,651
Non-trainable params: 0
________________________________________________

In [35]:
# Model Training

num_epochs = 10
history = model.fit_generator(train_generator, 
                              steps_per_epoch= train_generator.samples // batch_size, 
                              validation_data = validation_generator,
                              validation_steps = validation_generator.samples// batch_size, 
                              epochs = num_epochs)